## Wonderful!!!
#!pip install jupyter-dash

# My submission 

In [2]:
# Import required libraries
import pandas as pd
import dash
#import dash_html_components as html
#import dash_core_components as dcc
from dash import dcc,html
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
import plotly.express as px
from dash import no_update

In [3]:
# Read the airline data into pandas dataframe
df =  pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv')
#df.info()
df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [4]:
site_pie_df=df.groupby(['Launch Site'])['class'].mean().reset_index()
site_pie_df

,Launch Site,class
0,CCAFS LC-40,0.269231
1,CCAFS SLC-40,0.428571
2,KSC LC-39A,0.769231
3,VAFB SLC-4E,0.400000


In [7]:
scatter_payload_df=df.groupby(['Booster Version Category'])['class'].mean().reset_index()
#scatter_payload_df['class'].mean() # 0.45575757575757575
#df['class'].mean()               # 0.42857142857142855 --right
scatter_payload_df

,Booster Version Category,class
0,B4,0.545455
1,B5,1.000000
2,FT,0.666667
3,v1.0,0.000000
4,v1.1,0.066667


In [8]:
#df['Payload Mass (kg)'].unique() #48
df['Payload Mass (kg)'].max() #9600
df['Payload Mass (kg)'].min() # 0

0.0

In [ ]:
site_list=df['Launch Site'].unique().tolist()
site_list

In [ ]:
# Import required libraries
import pandas as pd
import dash
#import dash_html_components as html
#import dash_core_components as dcc
from dash import Input, Output, html, dcc
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown', 

                                            options=[
                                                {'label':'allSite' , 'value':'ALL' } ,
                                                {'label':'CCAFS LC-40' , 'value':'CCAFS LC-40' } ,
                                                {'label':'VAFB SLC-4E' , 'value':'VAFB SLC-4E' } ,
                                                {'label':'KSC LC-39A' , 'value':'KSC LC-39A' } ,
                                                {'label':'CCAFS SLC-40' , 'value':'CCAFS SLC-40' } ,
                                                    ],
                                                    value='ALL',
                                            placeholder="allSite",
                                            searchable=True,
                                            style={'width':'80%', 'padding':'3px', 'font-size': '20px', 
                                            'text-align-last' : 'center'},),

                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',
                                        min=0, max=10000, step=1000,
                                        marks={0: '0', 100: '100'},
                                        value=[min_payload, max_payload]),
                                

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),  # output id respect the outlayer id.
    Input(component_id='site-dropdown', component_property='value'),
    )
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(spacex_df, values='class',
        names='class', 
        title=f'Landing Rate of {entered_site} Site')
        return fig
    else:
        df=spacex_df.loc[spacex_df['Launch Site']==entered_site]['class'].value_counts().reset_index(),
        fig = px.pie(df, values='class',
        names='class', 
        title=f'Landing Rate of {entered_site} Site')
        return fig
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value'),
    Input(component_id='payload-slider',component_property='value'),
)
def get_payload_scatter_chart(entered_site,slider_range):   # the paramter is combine the dropdown,and slider_range, how many input respect the paramter.
    filtered_df = spacex_df
    low, high = slider_range
    mask = (df['petal_width'] >int(low)) & (df['petal_width'] < int(high))
    df = px.data.iris() # iris is a pandas DataFrame
    if entered_site == 'ALL':
        pldf=spacex_df.loc[mask]
        df = px.data.pldf()
        fig= px.scatter(spacex_df,y='class',x='Payload Mass (kg)',color='Booster Version Category'
        names='f{entered_site} Success Launch',
        title=f'Landing Rate of {low} to {high} ' ' {entered_site} Site')              )
        return fig

    else:
        df=spacex_df.loc[spacex_df['Launch Site']==entered_site]
        pldf=df.loc[mask]
        df = px.data.pldf()
        fig = px.catterie(df, y='class',x='Payload Mass (kg)', color='Booster Version Category',
        names='f{entered_site} Success Launch',
        title=f'Landing Rate of {low} to {high} ' ' {entered_site} Site')
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server()

# From classmate1 Mario Saraiva as an instance

In [3]:
#Author: Mario Saraiva
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px
# Read the airline data into pandas dataframe
spacex_df =pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv')

max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [4]:
spacex_df.head(2)

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0


In [ ]:
# Create a dash application
app = dash.Dash(__name__)
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # Dropdown
                                dcc.Dropdown(id='site_dropdown',
                                          options=[
                                              {'label': 'All Sites', 'value': 'ALL'},
                                              {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                              {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                              {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                              {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},],
                                           value= 'ALL',
                                           placeholder='Select a Launch Site here',
                                           searchable = True),    
                                
                                html.Br(),
                                # Pie Chart
                                html.Div(dcc.Graph(id='success_pie_chart')),
                                
                                html.Br(),
                                html.P("Payload range (Kg):"),
                                #Slider
                                dcc.RangeSlider(id='payload_slider',
                                    min=0, max=10000, step=1000, 
                                    marks={0: '0',
                                    100: '100'},
                                    value=[min_payload, max_payload]),
                                html.Div(dcc.Graph(id='success_payload_scatter_chart')),
                                ])
@app.callback([
    Output(component_id='success_pie_chart', component_property='figure'),
    Output(component_id='success_payload_scatter_chart', component_property='figure'),
    Input(component_id='site_dropdown', component_property='value'),
    Input(component_id='payload_slider', component_property='value')]
    
              )
def get_pie_chart(entered_site, slider_input):
    filtered_df = spacex_df
    
    #Pie chart
    filtered_df['dummy'] = 1
    if entered_site == 'ALL':
        fig = px.pie(filtered_df, values='dummy', names='class', title='Launch Success Rate')
        success_pie_chart = fig
        
    else:
        filtered_df = filtered_df[ filtered_df['Launch Site'] == entered_site ]
        fig = px.pie(filtered_df, values='dummy', names='class', title='Launch Success Rate')
        success_pie_chart = fig
    
    #Scatter chart    
        f_df = filtered_df[filtered_df['Payload Mass (kg)'] >= float(slider_input[0])]
        f_df = f_df[f_df['Payload Mass (kg)'] <= float(slider_input[1])]

        scatter_fig = px.scatter(f_df, x='Payload Mass (kg)', y='class', 
        color='Booster Version Category', title='Success rate and Payload Mass (kg)')

        success_payload_scatter_chart = scatter_fig


    
        
          
    

        
    
    
  
    return[success_pie_chart, success_payload_scatter_chart]
    
# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Jan/2023 17:04:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2023 17:04:35] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [15/Jan/2023 17:04:35] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 304 -
127.0.0.1 - - [15/Jan/2023 17:04:35] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [15/Jan/2023 17:04:35] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [15/Jan/2023 17:04:35] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2023 17:04:35] "GET /_dash-dependencies HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/shuangtian/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/shuangtian/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/shuangtian/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/shuangtian/opt/anaconda3/lib/python3.9/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/shuangtian/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/shuangtian/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  Fil

127.0.0.1 - - [15/Jan/2023 17:04:36] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [15/Jan/2023 17:04:36] "GET /_favicon.ico?v=2.5.1 HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2023 17:04:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2023 17:04:54] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [15/Jan/2023 17:04:54] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 304 -
127.0.0.1 - - [15/Jan/2023 17:04:54] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [15/Jan/2023 17:04:54] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [15/Jan/2023 17:04:54] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2023 17:04:54] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2023 17:04:54] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [15/Jan/2023 17:04:54] "

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/shuangtian/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/shuangtian/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/shuangtian/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/shuangtian/opt/anaconda3/lib/python3.9/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/shuangtian/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/shuangtian/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  Fil

...

127.0.0.1 - - [15/Jan/2023 17:05:35] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [15/Jan/2023 17:05:42] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/shuangtian/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/shuangtian/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/shuangtian/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/shuangtian/opt/anaconda3/lib/python3.9/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/shuangtian/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/shuangtian/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  Fil

127.0.0.1 - - [15/Jan/2023 17:06:07] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [15/Jan/2023 17:06:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2023 17:06:12] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2023 17:30:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2023 17:30:56] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [15/Jan/2023 17:30:56] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [15/Jan/2023 17:30:56] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 304 -
127.0.0.1 - - [15/Jan/2023 17:30:56] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [15/Jan/2023 17:30:56] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2023 17:30:56] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2023 17:30:56] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
1

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/shuangtian/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/shuangtian/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/shuangtian/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/shuangtian/opt/anaconda3/lib/python3.9/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/shuangtian/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/shuangtian/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  Fil

" 304 -
127.0.0.1 - - [15/Jan/2023 17:30:56] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [15/Jan/2023 17:30:56] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [15/Jan/2023 17:31:01] "POST /_dash-update-component HTTP/1.1" 200 -


# From classmate2 kurasahakai as an instance

In [ ]:
# Import required libraries
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df =pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv')

max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                             options=[
                                                    {'label': 'All Sites', 'value': 'ALL'},
                                                    {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                    {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                    {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                    {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}],
                                             value='ALL',
                                             placeholder="State",
                                             searchable=True
                                            ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0: '0', 2500: '2500', 5000:"5000",7500:"7500",10000:"10000"},
                                                value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie(launch_site):    
    if launch_site == 'ALL':
        fig = px.pie(spacex_df, values='class', names='Launch Site', title='Total Success Launches By Site')        
    else:
        selection_df = spacex_df[spacex_df["Launch Site"]==launch_site].groupby("class").count().reset_index()
        fig= px.pie(selection_df, values="Unnamed: 0", names="class",title="Total Launches for site {}".format(launch_site))
    return fig
        
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'), 
              Input(component_id="payload-slider", component_property="value"))
def get_scatter(launch_site, payload_range):
    if launch_site == 'ALL':
        selection_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= int(payload_range[0])) & (spacex_df['Payload Mass (kg)'] <= int(payload_range[1]))]
        fig = px.scatter(selection_df, x='Payload Mass (kg)', y='class', color='Booster Version Category', title="Correlation between payload and success for all sites")
    else:
        selection_df = spacex_df[(spacex_df['Launch Site'] == launch_site) & (spacex_df['Payload Mass (kg)'] >= int(payload_range[0])) & (spacex_df['Payload Mass (kg)'] <= int(payload_range[1]))]
        fig = px.scatter(selection_df, x='Payload Mass (kg)', y='class', color='Booster Version Category', title="Correlation between payload and success for site {}".format(launch_site))
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)